In [19]:
import requests
import pandas as pd

In [2]:
# Where to get the data of the resource
url = 'https://opendata-ajuntament.barcelona.cat/data/api/action/datastore_search?resource_id=290eb517-e7fa-41fb-aa59-389becb8f55b'

In [8]:
r = requests.get(url)
assert r.status_code == 200, "Request was not succesful, Status Code: {code}".format(code=r.status_code)

AssertionError: Request was not succesful, Status Code: 503

In [4]:
# Decapsulating the json object into a DataFrame object
bcn_covid_measures = r.json()['result']['records']
bcn_covid_df = pd.DataFrame(bcn_covid_measures)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [11]:
bcn_covid_df.head()

NameError: name 'bcn_covid_df' is not defined

In [10]:
import sqlite3
from sqlalchemy import create_engine
import os

In [22]:
database_name = 'opendatabcn.db'

# If the DDBB do not exist in our WD we create it
if database_name not in os.listdir():
    print('Creating Database object:')
    
    conn = sqlite3.connect(database_name)
    conn.close()
    
    assert database_name in os.listdir(), "Database could not be created."
    print(f"Database {database_name} created succesfully")
    
else:
    print(f"{database_name} already in directory")

opendatabcn.db already in directory


In [24]:
table = 'rep_measures_covid'

drops = """
DROP TABLE IF EXISTS {table};
""".format(table=table)

creates = """
CREATE TABLE {table} (
  test1 INT ,
  test2 CHAR(5) NOT NULL UNIQUE,
  test3 DATE NOT NULL,
  CONSTRAINT pk_rep_measures_covid PRIMARY KEY(test1)
)
""".format(table=table)

orders = (drops, creates)

In [25]:
conn = sqlite3.connect(database_name)
cursor = conn.cursor()

for order in orders:
    cursor.execute(order)
    conn.commit()
    
conn.close()

In [27]:
engine = create_engine(f"sqlite:///{database_name}")

try:
    bcn_covid_df.to_sql(table,
                        engine,
                        chunk_size=1000)
except Exception:
    raise Exception(f"Couldn't write pandas' DataFrame into table {table} of {database_name}")

Exception: Couldn't write pandas' DataFrame into table rep_measures_covid of opendatabcn.db